### imports and globals

In [18]:
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv
import pandas as pd

In [155]:
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAI

from enum import Enum, auto
from strenum import LowercaseStrEnum

from langchain.prompts import PromptTemplate
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.output_parsers import JsonOutputParser


# rfrom langchain_google_genai.chat_models import ChatGoogleGenerativeAI


In [156]:
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
OPEN_AI_KEY = os.environ["OPEN_AI_KEY"]

### prepare promt template


In [92]:
# Define your desired Pandas DataFrame.
df = pd.DataFrame(
    columns = ['Meal ID',
               'Date',
               'Time',
               'Meal component',
               'Quantity per serving',
               'Carbohydrates',
               'Protein',
               'Fat',
               'Calories'],
    data = [['1',
            '2025-02-25',
            '12:00',
            'Brown Bread (2 slices)',
            '2 slices',
            '20',
            '5',
            '5',
            '200']]
)

parser = PandasDataFrameOutputParser(dataframe=df)
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [157]:
nutri_prompt_template = ("You are a search assistant. The human user will give you a description of their meals, portion size and the time they consumed it. Your task is to analyse the meals, log the date and time of each meal,"
    "and then decompose the meal and its ingredients into their nutrient content. Use google search tool to get nutrient data from https://www.fatsecret.com/calories-nutrition/ if possible. MAke sensible guesses about portions where user does not provide"
    "Output the amount of carbohydrates, fats, proteins in grams, and calories. "
    "Please give a unique id to each meal, that is repeated across its components."
    "Answer with the nutritional breakdown of the meal in a json format (and nothing else) as "
    "[\{{\'Column1': 'Value1', 'Column2': 'Value2', 'Column3': 'Value3'\}},"
    "\{{\'Column1': 'Value4', 'Column2': 'Value5', 'Column3': 'Value6'\}}]"
    "The following are the key/column names, and you must extract the value from the meal analysis"
     "- meal_id"
    "- date"
    "- time"
    "- meal_component"
    "- quantity_per_serving"
    "- carbohydrates"
    "- protein"
    "- fat"
    "- calories"
    "Only answer with the json key-value list, and nothing else.")

In [161]:
parser = JsonOutputParser()
parser.get_format_instructions()

'Return a JSON object.'

In [162]:
# Set up the prompt.
prompt_template = PromptTemplate(
    template = nutri_prompt_template + "{format_instructions}" + "\n{meal_description}\n",
    input_variables=["meal_description"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [208]:
meal_description = ("today is 24th feb, I ate a cheeseburger with some slaw for lunch, and then a plain dosa for dinner, and yesterday I ate chowmein at 5pm")

# Generate the prompt
filled_prompt = prompt_template.invoke({"meal_description": meal_description})
filled_prompt

StringPromptValue(text="You are a search assistant. The human user will give you a description of their meals, portion size and the time they consumed it. Your task is to analyse the meals, log the date and time of each meal,and then decompose the meal and its ingredients into their nutrient content. Use google search tool to get nutrient data from https://www.fatsecret.com/calories-nutrition/ if possible. MAke sensible guesses about portions where user does not provideOutput the amount of carbohydrates, fats, proteins in grams, and calories. Please give a unique id to each meal, that is repeated across its components.Answer with the nutritional breakdown of the meal in a json format (and nothing else) as [\\{'Column1': 'Value1', 'Column2': 'Value2', 'Column3': 'Value3'\\},\\{'Column1': 'Value4', 'Column2': 'Value5', 'Column3': 'Value6'\\}]The following are the key/column names, and you must extract the value from the meal analysis- meal_id- date- time- meal_component- quantity_per_ser

### access GenAI chat API

In [209]:
class MODEL_ID(LowercaseStrEnum):
    GEMINI_1_5_FLASH_8B = "gemini-1.5-flash-8b"
    GEMINI_2_0_FLASH = "gemini-2.0-flash"
    GEMINI_1_5_FLASH = "gemini-1.5-flash"


In [210]:

# Define custom GenerateContentConfig parameters
client_options = {
    "temperature": 0.0,  # Controls randomness
    "max_output_tokens": 1000,  # Limit on output tokens
    "stop_sequences": ["\n"],   # Optional stop sequences
}


# Initialize ChatGoogleGenerativeAI with client options
llm = ChatGoogleGenerativeAI(
    model = MODEL_ID.GEMINI_1_5_FLASH,
    google_api_key = GOOGLE_API_KEY,
    max_output_tokens = 1000,
    temperature=0,
    top_p=0.95,
    top_k=20,
    config=GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    ))
   #client_options =  {'generate_content_config': {'max_output_tokens': 100}})

# Invoke the model with a prompt
response = llm.invoke(filled_prompt.text )
print(response.content)


```json
[
  {
    "meal_id": "20240223_1",
    "date": "2024-02-23",
    "time": "17:00",
    "meal_component": "Chowmein (estimated 1 serving)",
    "quantity_per_serving": "1 serving",
    "carbohydrates": "40",
    "protein": "10",
    "fat": "15",
    "calories": "350"
  },
  {
    "meal_id": "20240224_1",
    "date": "2024-02-24",
    "time": "12:00",
    "meal_component": "Cheeseburger",
    "quantity_per_serving": "1",
    "carbohydrates": "35",
    "protein": "25",
    "fat": "30",
    "calories": "450"
  },
  {
    "meal_id": "20240224_1",
    "date": "2024-02-24",
    "time": "12:00",
    "meal_component": "Slaw (estimated 1/2 cup)",
    "quantity_per_serving": "0.5 cup",
    "carbohydrates": "5",
    "protein": "1",
    "fat": "1",
    "calories": "25"
  },
  {
    "meal_id": "20240224_2",
    "date": "2024-02-24",
    "time": "20:00",
    "meal_component": "Plain Dosa (estimated 1)",
    "quantity_per_serving": "1",
    "carbohydrates": "30",
    "protein": "5",
    "fat": 

In [211]:
response

AIMessage(content='```json\n[\n  {\n    "meal_id": "20240223_1",\n    "date": "2024-02-23",\n    "time": "17:00",\n    "meal_component": "Chowmein (estimated 1 serving)",\n    "quantity_per_serving": "1 serving",\n    "carbohydrates": "40",\n    "protein": "10",\n    "fat": "15",\n    "calories": "350"\n  },\n  {\n    "meal_id": "20240224_1",\n    "date": "2024-02-24",\n    "time": "12:00",\n    "meal_component": "Cheeseburger",\n    "quantity_per_serving": "1",\n    "carbohydrates": "35",\n    "protein": "25",\n    "fat": "30",\n    "calories": "450"\n  },\n  {\n    "meal_id": "20240224_1",\n    "date": "2024-02-24",\n    "time": "12:00",\n    "meal_component": "Slaw (estimated 1/2 cup)",\n    "quantity_per_serving": "0.5 cup",\n    "carbohydrates": "5",\n    "protein": "1",\n    "fat": "1",\n    "calories": "25"\n  },\n  {\n    "meal_id": "20240224_2",\n    "date": "2024-02-24",\n    "time": "20:00",\n    "meal_component": "Plain Dosa (estimated 1)",\n    "quantity_per_serving": "1",

In [212]:
parsed_json = parser.parse(response.content)


In [229]:
df = pd.DataFrame.from_records(parsed_json)
df.groupby('date').cumcount() + 1
df

,meal_id,date,time,meal_component,quantity_per_serving,carbohydrates,protein,fat,calories
0,20240223_1,2024-02-23,17:00,Chowmein (estimated 1 serving),1 serving,40,10,15,350
1,20240224_1,2024-02-24,12:00,Cheeseburger,1,35,25,30,450
2,20240224_1,2024-02-24,12:00,Slaw (estimated 1/2 cup),0.5 cup,5,1,1,25
3,20240224_2,2024-02-24,20:00,Plain Dosa (estimated 1),1,30,5,5,200


### try google gemini directly

In [80]:
client = genai.Client(api_key=GOOGLE_API_KEY)

google_search_tool = Tool(
    google_search = GoogleSearch()
)

response_direct = client.models.generate_content(
    model=MODEL_ID.GEMINI_2_0_FLASH,
    contents=filled_prompt, 
    config=types.GenerateContentConfig(
        tools=[google_search_tool],
         response_modalities=["TEXT"],
    )
)


In [89]:
print(response_direct_text)

| Meal ID | Date       | Time  | Meal component | Quantity per serving | Carbohydrates (g) | Protein (g) | Fat (g) | Calories |
| ------- | ---------- | ----- | --------------- | -------------------- | ----------------- | ----------- | ------- | -------- |
| 1       | 2025-02-25 | 12:00 | Brown Bread    | 2 slices             | 28                | 8           | 2       | 160      |
| 1       | 2025-02-25 | 12:00 | Tomato         | 50g                | 2                 | 0.5         | 0.1     | 10       |
| 1       | 2025-02-25 | 12:00 | Cheese         | 1 slice              | 1                 | 6           | 8       | 100      |
| 1       | 2025-02-25 | 12:00 | Cucumber       | 50g                | 2                 | 0.4         | 0.1     | 8        |
| 2       | 2025-02-25 | 21:00 | Roti           | 2                    | 60                | 8           | 2       | 320      |
| 2       | 2025-02-25 | 21:00 | Chicken Curry  | 100g               | 8                 | 25          | 15

## agent

In [ ]:
from langchain.agents import initialize_agent, AgentType

# Combine LLM and tools into an agent
tools = [search_tool]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Example query that requires search
query = "What are the latest advancements in AI research? Please answer in not more than 5 lines"

response = agent.run(query)

print(response)


### postgres

In [202]:
f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'
df

,Meal ID,Date,Time,Meal component,Quantity per serving,Carbohydrates,Protein,Fat,Calories
0,1,2025-02-25,12:00,Brown Bread (2 slices),2 slices,20,5,5,200


In [243]:
import pandas as pd
from sqlalchemy import create_engine

# PostgreSQL connection details
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'
database = 'postgres'

# Create SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

# Create a DataFrame
#data = {'name': ['Alice', 'Bob', 'Charlie'], 'age': [25, 30, 35]}
#df = pd.DataFrame(data)

# Store DataFrame into PostgreSQL
df.to_sql('table_name', engine, index=False, if_exists='replace')


4

In [189]:
r		

In [244]:
df.to_sql('table_name', engine, index=False, if_exists='append')


4

In [247]:
df

,meal_id,date,time,meal_component,quantity_per_serving,carbohydrates,protein,fat,calories
0,20240223_1,2024-02-23,17:00,Chowmein (estimated 1 serving),1 serving,40,10,15,350
1,20240224_1,2024-02-24,12:00,Cheeseburger,1,35,25,30,450
2,20240224_1,2024-02-24,12:00,Slaw (estimated 1/2 cup),0.5 cup,5,1,1,25
3,20240224_2,2024-02-24,20:00,Plain Dosa (estimated 1),1,30,5,5,200


In [255]:
df['protein'].astype(int)

0    10
1    25
2     1
3     5
Name: protein, dtype: int64

In [249]:
df.groupby('date')['protein'].mean().reset_index()


TypeError: agg function failed [how->mean,dtype->object]